Python Trick:

A short Python code snippet meant as a
teaching tool. A Python Trick either teaches an aspect of
Python with a simple illustration, or it serves as a moti-
vating example, enabling you to dig deeper and develop
an intuitive understanding.

### Encapsulate main files with main function

### Underscores, Dunders, and More

1. Single Leading Underscore: “_var”

The underscore prefix is meant as a hint to tell another programmer
that a variable or method starting with a single underscore is intended
for internal use. This convention is defined in PEP 8, the most com-
monly used Python code style guide.8

In [ ]:
# my_module.py:
def external_func():
    return 23

def _internal_func():
return 42

In [ ]:
>>> from my_module import *
>>> external_func()
23
>>> _internal_func()
NameError: "name '_internal_func' is not defined"

By the way, wildcard imports should be avoided as they make it un-
clear which names are present in the namespace.10 It’s better to stick
to regular imports for the sake of clarity. Unlike wildcard imports, reg-
ular imports are not affected by the leading single underscore naming
convention:

In [ ]:
>>> import my_module
>>> my_module.external_func()
23
>>> my_module._internal_func()
42

2. Single Trailing Underscore: “var_”

Sometimes the most fitting name for a variable is already taken by a
keyword in the Python language. Therefore, names like class or def
cannot be used as variable names in Python. In this case, you can
append a single underscore to break the naming conflict:

In [ ]:
>>> def make_object(name, class):
SyntaxError: "invalid syntax"
>>> def make_object(name, class_):
...
pass

In summary, a single trailing underscore (postfix) is used by conven-
tion to avoid naming conflicts with Python keywords. This convention
is defined and explained in PEP 8.

3. Double Leading Underscore: “__var”

A double underscore prefix causes the Python interpreter to rewrite
the attribute name in order to avoid naming conflicts in subclasses. This is also called name mangling—the interpreter changes the name
of the variable in a way that makes it harder to create collisions when
the class is extended later.

In [ ]:
class Test:
    def __init__(self):
        self.foo = 11
        self._bar = 23
        self.__baz = 23

In [ ]:
>>> t = Test()
>>> dir(t)
['_Test__baz', '__class__', '__delattr__', '__dict__',
'__dir__', '__doc__', '__eq__', '__format__', '__ge__',
'__getattribute__', '__gt__', '__hash__', '__init__',
'__le__', '__lt__', '__module__', '__ne__', '__new__',
'__reduce__', '__reduce_ex__', '__repr__',
'__setattr__', '__sizeof__', '__str__',
'__subclasshook__', '__weakref__', '_bar', 'foo']

This gives us a list with the object’s attributes. Let’s take this list and
look for our original variable names foo, _bar, and __baz. I promise
you’ll notice some interesting changes.

First of all, the self.foo variable appears unmodified as foo in the
attribute list.

Next up, self._bar behaves the same way—it shows up on the class
as _bar. Like I said before, the leading underscore is just a convention
in this case—a hint for the programmer.
However, with self.__baz things look a little different. When you
search for __baz in that list, you’ll see that there is no variable with
that name.

So what happened to __baz?

If you look closely, you’ll see there’s an attribute called _Test__baz
on this object. This is the name mangling that the Python interpreter
applies. It does this to protect the variable from getting overridden in
subclasses.

Let’s create another class that extends the Test class and attempts to
override its existing attributes added in the constructor:

In [ ]:
class ExtendedTest(Test):
    def __init__(self):
        super().__init__()
        self.foo = 'overridden'
        self._bar = 'overridden'
        self.__baz = 'overridden'

In [ ]:
>>> t2 = ExtendedTest()
>>> t2.foo
'overridden'
>>> t2._bar
'overridden'
>>> t2.__baz
AttributeError:
"'ExtendedTest' object has no attribute '__baz'"

In [ ]:
>>> dir(t2)
['_ExtendedTest__baz', '_Test__baz', '__class__',
'__delattr__', '__dict__', '__dir__', '__doc__',
'__eq__', '__format__', '__ge__', '__getattribute__',
'__gt__', '__hash__', '__init__', '__le__', '__lt__',
'__module__', '__ne__', '__new__', '__reduce__',
'__reduce_ex__', '__repr__', '__setattr__',
'__sizeof__', '__str__', '__subclasshook__',
'__weakref__', '_bar', 'foo', 'get_vars']

As you can see, __baz got turned into _ExtendedTest__baz to pre-
vent accidental modification. But the original _Test__baz is also still
around:

In [ ]:
>>> t2._ExtendedTest__baz
'overridden'
>>> t2._Test__baz
42

4. Double Leading and Trailing Underscore: “__var__”

Perhaps surprisingly, name mangling is not applied if a name starts
and ends with double underscores. Variables surrounded by a double
underscore prefix and postfix are left unscathed by the Python inter-
preter:

In [ ]:
class PrefixPostfixTest:
    def __init__(self):
        self.__bam__ = 42
>>> PrefixPostfixTest().__bam__
42

However, names that have both leading and trailing double under-
scores are reserved for special use in the language. This rule covers
things like __init__ for object constructors, or __call__ to make ob-
jects callable.
These dunder methods are often referred to as magic methods—but
many people in the Python community, including myself, don’t like
that word. It implies that the use of dunder methods is discouraged,
which is entirely not the case. They’re a core feature in Python and
should be used as needed. There’s nothing “magical” or arcane about
them.
However, as far as naming conventions go, it’s best to stay away from
using names that start and end with double underscores in your own
programs to avoid collisions with future changes to the Python lan-
guage.

### Asserts

- Python’s assert statement is a debugging aid that tests a condi-
tion as an internal self-check in your program.
- Asserts should only be used to help developers identify bugs.
They’re not a mechanism for handling run-time errors.
- Asserts can be globally disabled with an interpreter setting.

I cannot recommend using assertions in production code under normal circumstances. While there are rare edge cases where it might be tempting, the drawbacks often outweigh the benefits. Here's why:

Reasons to avoid assertions in production:

Can be disabled: Most languages allow disabling assertions in production builds, rendering them useless for catching errors at runtime.
Don't provide meaningful feedback: Assertions typically just crash the program with a generic message, making debugging harder.
Mask deeper issues: Unexpected scenarios often indicate errors beyond the assertion point, hiding the actual root cause.
Performance impact: Even disabled assertions can have a minor performance overhead in production.
Alternatives to assertions in production:

Robust validation: Implement thorough validation logic at the entry points of your application, ensuring expected data formats and values.
Exception handling: For external errors, use exceptions to gracefully handle and report issues, providing informative messages for debugging.
Logging: Log suspicious conditions and potential problems for later analysis and investigation.
However, there might be a rare exceptional case where an assertion could be justified:

Critical invariant protection: If a violation of a specific internal state would lead to catastrophic consequences even in production (e.g., data corruption), you might consider an assertion as a last-resort safeguard. But remember, this should be a well-defined, highly unlikely scenario, and the assertion should provide a detailed error message for later analysis.
In summary: While exceptions have their own limitations, using well-designed exceptions combined with thorough validation is generally the recommended approach for error handling in production code. Avoid relying on assertions, as they offer little value and potential drawbacks in real-world applications.

In [ ]:
def delete_product(prod_id, user):
    assert user.is_admin(), 'Must be admin'
    assert store.has_product(prod_id), 'Unknown product'
    store.get_product(prod_id).delete()

In [ ]:
def delete_product(product_id, user):
    if not user.is_admin():
        raise AuthError('Must be admin to delete')
    if not store.has_product(product_id):
        raise ValueError('Unknown product id')
    store.get_product(product_id).delete()

### Context Managers and the with Statement

In [ ]:
with open('hello.txt', 'w') as f:
    f.write('hello, world!')

Opening files using the with statement is generally recommended be-
cause it ensures that open file descriptors are closed automatically af-
ter program execution leaves the context of the with statement. Inter-
nally, the above code sample translates to something like this:

In [ ]:
f = open('hello.txt', 'w')
try:
    f.write('hello, world')
finally:
    f.close()

You can already tell that this is quite a bit more verbose. Note that
the try...finally statement is significant. It wouldn’t be enough to
just write something like this:

In [ ]:
f = open('hello.txt', 'w')
f.write('hello, world')
f.close()

This implementation won’t guarantee the file is closed if there’s an ex-
ception during the f.write() call—and therefore our program might
leak a file descriptor. That’s why the with statement is so useful. It
makes properly acquiring and releasing resources a breeze.

### Python’s Functions Are First-Class

- Everything in Python is an object, including functions. You can
assign them to variables, store them in data structures, and pass
or return them to and from other functions (first-class func-
tions.)
- First-class functions allow you to abstract away and pass
around behavior in your programs.
- Functions can be nested and they can capture and carry some
of the parent function’s state with them. Functions that do this
are called closures.
- Objects can be made callable. In many cases this allows you to
treat them like functions.

In [10]:
def yell(text):
    return text.upper() + '!'
yell('hello')

'HELLO!'

#### Functions Are Objects

In [11]:
bark = yell

bark('woof')

'WOOF!'

In [12]:
id(bark)

139932361415696

In [13]:
id(yell)

139932361415696

Function objects and their names are two separate concerns. Here’s
more proof: You can delete the function’s original name (yell). Since
another name (bark) still points to the underlying function, you can
still call the function through it:

In [14]:
del yell

yell('hello?')

NameError: name 'yell' is not defined

In [15]:
id(yell)

NameError: name 'yell' is not defined

In [16]:
bark('hey')

'HEY!'

By the way, Python attaches a string identifier to every function at
creation time for debugging purposes. You can access this internal
identifier with the __name__ attribute:

In [17]:
bark.__name__

'yell'

#### Functions Can Be Stored in Data Structures

In [18]:
funcs = [bark, str.lower, str.capitalize]

In [19]:
funcs

[<function __main__.yell(text)>,
 <method 'lower' of 'str' objects>,
 <method 'capitalize' of 'str' objects>]

In [20]:
for f in funcs:
    print(f, f('hey there'))

<function yell at 0x7f448ab1dc10> HEY THERE!
<method 'lower' of 'str' objects> hey there
<method 'capitalize' of 'str' objects> Hey there


#### Functions Can Be Passed to Other Functions

In [22]:
def greet(func):
    greeting = func('Hi, I am a Python program')
    print(greeting)
greet(bark)

HI, I AM A PYTHON PROGRAM!


The classical example for higher-order functions in Python is the built-
in map function. It takes a function object and an iterable, and then
calls the function on each element in the iterable, yielding the results
as it goes along.

In [23]:
list(map(bark, ['hello', 'hey', 'hi']))

['HELLO!', 'HEY!', 'HI!']

#### Functions Can Be Nested

In [24]:
def speak(text):
    def whisper(t):
        return t.lower() + '...'
    return whisper(text)

speak('Hello, World')

'hello, world...'

Here’s the kicker though—whisper does not exist outside speak:

In [25]:
whisper('Yo')

NameError: name 'whisper' is not defined

In [26]:
speak.whisper

AttributeError: 'function' object has no attribute 'whisper'

But what if you really wanted to access that nested whisper function
from outside speak? Well, functions are objects—you can return the
inner function to the caller of the parent function.

In [28]:
def get_speak_func(volume):
    def whisper(text):
        return text.lower() + '...'
    def yell(text):
        return text.upper() + '!'

    if volume > 0.5:
        return yell
    else:
        return whisper

get_speak_func(0.3)

<function __main__.get_speak_func.<locals>.whisper(text)>

#### Functions Can Capture Local State

I’m going to slightly rewrite the previous get_speak_func example
to illustrate this. The new version takes a “volume” and a “text” argu-
ment right away to make the returned function immediately callable:

In [29]:
def get_speak_func(text, volume):
    def whisper():
        return text.lower() + '...'
    def yell():
        return text.upper() + '!'

    if volume > 0.5:
        return yell
    else:
        return whisper
get_speak_func('Hello, World', 0.7)()

'HELLO, WORLD!'

In [30]:
def make_adder(n):
    def add(x):
        return x + n
    return add

plus_3 = make_adder(3)
plus_5 = make_adder(5)

In [31]:
plus_3(4)

7

In [32]:
plus_5(4)

9

#### Objects Can Behave Like Functions

If an object is callable it means you can use the round parentheses
function call syntax on it and even pass in function call arguments.
This is all powered by the __call__ dunder method. Here’s an exam-
ple of class defining a callable object:

In [34]:
class Adder:
    def __init__(self, n):
        self.n = n

    def __call__(self, x):
        return self.n + x

plus_3 = Adder(3)
plus_3(4)

7

Of course, not all objects will be callable. That’s why there’s a built-in
callable function to check whether an object appears to be callable
or not:

In [35]:
callable(plus_3)

True

In [37]:
callable('hello')

False

### Lambdas Are Single-Expression Functions

The lambda keyword in Python provides a shortcut for declaring
small anonymous functions. Lambda functions behave just like
regular functions declared with the def keyword. They can be used
whenever function objects are required.

In [38]:
add = lambda x, y: x + y
add(5, 3)

8

You could declare the same add function with the def keyword, but it
would be slightly more verbose:

In [39]:
def add(x, y):
    return x + y

add(5, 3)

8

Now you might be wondering, “Why the big fuss about lambdas? If
they’re just a slightly more concise version of declaring functions with
def, what’s the big deal?”

Take a look at the following example and keep the words function ex-
pression in your head while you do that:

In [40]:
(lambda x, y: x + y)(5, 3)

8

There’s another syntactic difference between lambdas and regular
function definitions. Lambda functions are restricted to a single
expression. This means a lambda function can’t use statements or
annotations—not even a return statement.

How do you return values from lambdas then? Executing a lambda
function evaluates its expression and then automatically returns
the expression’s result, so there’s always an implicit return state-
ment. That’s why some people refer to lambdas as single expression
functions.

#### Lambdas You Can Use

When should you use lambda functions in your code? Technically, any
time you’re expected to supply a function object you can use a lambda
expression. And because lambdas can be anonymous, you don’t even
need to assign them to a name first.

This can provide a handy and “unbureaucratic” shortcut to defining a
function in Python. My most frequent use case for lambdas is writing
short and concise key funcs for sorting iterables by an alternate key:

In [41]:
tuples = [(1, 'd'), (2, 'b'), (4, 'a'), (3, 'c')]
sorted(tuples, key=lambda x: x[1])

[(4, 'a'), (2, 'b'), (3, 'c'), (1, 'd')]

In [42]:
sorted(range(-5, 6), key=lambda x: x * x)

[0, -1, 1, -2, 2, -3, 3, -4, 4, -5, 5]

#### But Maybe You Shouldn’t…

For example, doing something like this to save two lines of code is just
silly. Sure, technically it works and it’s a nice enough “trick.” But it’s
also going to confuse the next gal or guy that has to ship a bugfix under
a tight deadline:

In [44]:
class Car:
    rev = lambda self: print('Wroom!')
    crash = lambda self: print('Boom!')

my_car = Car()
my_car.crash()

Boom!


I have similar feelings about complicated map() or filter() con-
structs using lambdas. Usually it’s much cleaner to go with a list
comprehension or generator expression:

In [45]:
# Harmful:
list(filter(lambda x: x % 2 == 0, range(16)))

[0, 2, 4, 6, 8, 10, 12, 14]

In [46]:
# Better:
[x for x in range(16) if x % 2 == 0]

[0, 2, 4, 6, 8, 10, 12, 14]

### The Power of Decorators

Any sufficiently generic functionality you can tack on to an existing
class or function’s behavior makes a great use case for decoration.
This includes the following:
- logging
- enforcing access control and authentication
- instrumentation and timing functions
- rate-limiting
- caching, and more

#### Python Decorator Basics

Now, what are decorators really? They “decorate” or “wrap” another
function and let you execute code before and after the wrapped func-
tion runs.

In
basic terms, a decorator is a callable that takes a callable as input and
returns another callable.

In [47]:
def null_decorator(func):
    return func

As you can see, null_decorator is a callable (it’s a function), it takes
another callable as its input, and it returns the same input callable
without modifying it.

Let’s use it to decorate (or wrap) another function:

In [49]:
def greet():
    return 'Hello!'

greet = null_decorator(greet)
greet()

'Hello!'

In [50]:
@null_decorator
def greet():
    return 'Hello!'

greet()

'Hello!'

Here’s a slightly more complex decorator which converts the result of
the decorated function to uppercase letters:

In [54]:
def uppercase(func):
    def wrapper():
        original_result = func()
        modified_result = original_result.upper()
        return modified_result
    return wrapper

@uppercase
def greet():
    return 'Hello!'
greet()

'HELLO!'

In [61]:
def uppercase(func):
    modified_result = func().upper()
    return modified_result

@uppercase
def greet():
    return 'Hello!'
greet()

TypeError: 'str' object is not callable

In [63]:
def strong(func):
    def wrapper():
        return '<strong>' + func() + '</strong>'
    return wrapper

def emphasis(func):
    def wrapper():
        return '<em>' + func() + '</em>'
    return wrapper

@strong
@emphasis
def greet():
    return 'Hello!'

greet()

'<strong><em>Hello!</em></strong>'

This clearly shows in what order the decorators were applied: from
bottom to top. First, the input function was wrapped by the @emphasis
decorator, and then the resulting (decorated) function got wrapped
again by the @strong decorator.

To help me remember this bottom to top order, I like to call this be-
havior decorator stacking. You start building the stack at the bottom
and then keep adding new blocks on top to work your way upwards.

If you break down the above example and avoid the @ syntax to apply
the decorators, the chain of decorator function calls looks like this:

In [64]:
decorated_greet = strong(emphasis(greet))

As a best practice, I’d recommend that you use functools.wraps in
all of the decorators you write yourself. It doesn’t take much time and
it will save you (and others) debugging headaches down the road.

In [ ]:
import functools

def uppercase(func):
    @functools.wraps(func)
    def wrapper():
        return func().upper()
    return wrapper

### Fun With *args and **kwargs

So what are *args and **kwargs parameters used for? They allow a
function to accept optional arguments, so you can create flexible APIs
in your modules and classes:

In [ ]:
def foo(required, *args, **kwargs):
    print(required)
    if args:
        print(args)
    if kwargs:
        print(kwargs)